<a href="https://colab.research.google.com/github/Arhan2002/BloodBank-Management-Project/blob/main/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import re
import pandas as pd
import numpy as np

from collections import Counter
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# Extract Time
def date_time(s):
    pattern = '^([0-9]+)(\/)([0-9]+)(\/)([0-9]+), ([0-9]+):([0-9]+)[ ]?(AM|PM|am|pm)? -'
    result = re.match(pattern, s)
    if result:
        return True
    return False

# Find Authors or Contacts
def find_author(s):
    s = s.split(":")
    if len(s)==2:
        return True
    else:
        return False

# Finding Messages
def getDatapoint(line):
    splitline = line.split(' - ')
    dateTime = splitline[0]
    date, time = dateTime.split(", ")
    message = " ".join(splitline[1:])
    if find_author(message):
        splitmessage = message.split(": ")
        author = splitmessage[0]
        message = " ".join(splitmessage[1:])
    else:
        author= None
    return date, time, author, message

In [6]:
data = []
conversation = 'WhatsApp Chat with Hello desis.txt'
with open(conversation, encoding="utf-8") as fp:
    fp.readline()
    messageBuffer = []
    date, time, author = None, None, None
    while True:
        line = fp.readline()
        if not line:
            break
        line = line.strip()
        if date_time(line):
            if len(messageBuffer) > 0:
                data.append([date, time, author, ' '.join(messageBuffer)])
            messageBuffer.clear()
            date, time, author, message = getDatapoint(line)
            messageBuffer.append(message)
        else:
            messageBuffer.append(line)

In [11]:
pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 10.3 MB/s 


In [12]:
df = pd.DataFrame(data, columns=["Date", 'Time', 'Author', 'Message'])
df['Date'] = pd.to_datetime(df['Date'])

data = df.dropna()
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
#from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiments = SentimentIntensityAnalyzer()
data["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data["Message"]]
data["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data["Message"]]
data["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data["Message"]]
print(data.head())


        Date   Time        Author  \
2 2022-03-14  21:33      Maitreyi   
3 2022-03-14  21:33  Shreya Pawar   
4 2022-03-14  21:33  Shreya Pawar   
5 2022-03-14  21:33  Shreya Pawar   
6 2022-03-14  21:33  Shreya Pawar   

                                             Message  Positive  Negative  \
2  Aj jisne b motivate Kiya baal katwane ke liye ...     0.261       0.0   
3                                       Bhai tu kywa     0.000       0.0   
4                                            Ktwa!!!     0.000       0.0   
5                              You will look hawtt!!     0.000       0.0   
6                                      Hawt and secy     0.000       0.0   

   Neutral  
2    0.739  
3    1.000  
4    1.000  
5    1.000  
6    1.000  


In [14]:
x=sum(data["Positive"])
y=sum(data["Negative"])
z=sum(data["Neutral"])

def score(a,b,c):
    if (a>b) and (a>c):
        print("Positive ")
    if (b>a) and (b>c):
        print("Negative")
    if (c>a) and (c>b):
        print("Neutal")

score(x,y,z)

Neutal
